<a href="https://colab.research.google.com/github/nitinmadan7468/Google_Colab_Code/blob/main/Google_Colab_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#without Impressions
target =pd.DataFrame() # declare an empty dataframe
from fbprophet import Prophet # import the prophet library
from matplotlib import pyplot as plt
for i in Kampagnen.groups: # for loop to use the generated time series for each campaign
  group = Kampagnen.get_group(i)
  train_ind = int(len(group)*0.75) # training and testing data, using 75% for training
  train = group[:train_ind]
  test = group[train_ind:]
  m=Prophet(growth='linear',weekly_seasonality=True,daily_seasonality=True,yearly_seasonality=True)#adding seasonality for daily data
  m.add_country_holidays(country_name='DE')#add country holidays
  #corr_clicks =group['y'].corr(group['Clicks'])
  #corr_Imp =group['y'].corr(group['Impressions'])
  #corr_spent =group['y'].corr(group['spent']
  #if (corr_clicks > 0.5 AN corr_spent > 0.5) AND/OR (cond3 AND/OR cond4):  
   # m=Prophet(growth='linear',weekly_seasonality=True,daily_seasonality=True,yearly_seasonality=True)
    #m.add_country_holidays(country_name='DE')                         
  m.add_regressor('Clicks') #adding respective regressors Clicks, spent, Impressions
  m.add_regressor('spent')
  #m.add_regressor('Impressions')
  #else:  
   # m=Prophet(growth='linear',weekly_seasonality=True,daily_seasonality=True,yearly_seasonality=True)
    #m.add_country_holidays(country_name='DE') 
  #m.add_regressor('Impressions')
  m.fit(group)
  future =m.make_future_dataframe(periods=test.shape[0])
  forecast=m.predict(group)
  #print("campaign_name: %s" %group.Campaign.head(1))
  forecast=forecast[['ds','yhat','yhat_lower','yhat_upper']]
  forecast=group.merge(forecast, left_on=['ds'], right_on = ['ds'], how='left')
  fig1 = m.plot(forecast, xlabel=group['Campaign'].unique(),plot_cap=True,uncertainty= True, figsize= (16,12))
  #results=forecast_Mob_69.copy()
 #marking points which have an average greater than the mean average of the last 30 days        
  forecast['rolling_Clicks_mean_30'] = forecast.Clicks.shift().rolling(30).mean()
  forecast['rolling_ctr_mean_30'] = forecast.y.shift().rolling(30).mean()
  forecast['rolling_yhat_mean_30'] = forecast.yhat.shift().rolling(30).mean()
  forecast['rolling_spent_mean_30'] = forecast.spent.shift().rolling(30).mean()
  forecast['rolling_Imp_mean_30'] = forecast.Impressions.shift().rolling(30).mean()
  #forecast['rolling_Clicks_mean_7'] = forecast.Clicks.shift().rolling(7).mean()
  #forecast['rolling_ctr_mean_7'] = forecast.y.shift().rolling(7).mean()
  #forecast['rolling_yhat_mean_7'] = forecast.yhat.shift().rolling(7).mean()
  target=target.append(forecast)

In [ ]:
results = target.copy()
def detect_anomalies(results):
    forecasted = results[['Campaign','ds', 'yhat', 'yhat_lower', 'yhat_upper','rolling_ctr_mean_30', 'y']].copy()
    

    forecasted['anomaly'] = 0
    forecasted.loc[forecasted['y'] > forecasted['yhat_upper'], 'anomaly'] = 1
    forecasted.loc[forecasted['y'] < forecasted['yhat_lower'], 'anomaly'] = -1

    #anomaly importances
    '''
    forecasted['importance'] = 0
    
    forecasted.loc[forecasted['anomaly'] ==1, 'importance'] = \
        (forecasted['y'] - forecasted['yhat_upper'])/forecast['y']
    forecasted.loc[forecasted['anomaly'] ==-1, 'importance'] = \
        (forecasted['yhat_lower'] - forecasted['y'])/forecast['y']

    if forecasted['anamoly'] == 1:
      forecasted['importance'] =  (forecasted['y'] - forecasted['yhat_upper'])/forecast['y']
    elif  forecasted['anamoly'] == -1:
      (forecasted['yhat_lower'] - forecasted['y'])/forecast['y']
'''
    return forecasted

pred = detect_anomalies(results)
pred =pred[(pred['ds'] > '2021-05-01') & (pred['anomaly'].isin(['1','-1']))]
pred['30day_Anamoly'] = pred.apply(lambda x: 'low' if(np.abs(x['y']) >  1.25* x['rolling_ctr_mean_30'] and np.abs(x['y'])< 2.0 * x['rolling_ctr_mean_30'])  else ('Medium' if(np.abs(x['y']) > 2.0 * x['rolling_ctr_mean_30']) and np.abs(x['y'])< 3.0 * x['rolling_ctr_mean_30'] else ('High' if(np.abs(x['y']) > 3.0 * x['rolling_ctr_mean_30']) else ('no_change'))), axis=1) 
final_anamolies = pred[pred['30day_Anamoly'].isin(['low','Medium','High'])]
final_anamolies
print(final_anamolies.shape)